In [1]:
using CSV
using DataFrames
using Serialization
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210117.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
findmissing(x) = (ismissing(x) || x == -999999) ? missing : x;

In [6]:
data_weekly = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    
    :total_adult_patients_hospitalized_confirmed_covid_7_day_avg => ByRow(findmissing) => :total_occupancy,
    :staffed_icu_adult_patients_confirmed_covid_7_day_avg => ByRow(findmissing) => :icu_occupancy,
);
sort!(data_weekly, [:hospital, :hospital_id, :date]);

In [7]:
function latest_val(xs)
    xs = filter(x -> !ismissing(x), xs)
    z = length(xs) == 0 ? missing : xs[end]
    z = coalesce(z, 0)
    return z
end;

In [8]:
data_latest = combine(groupby(data_weekly, [:hospital, :hospital_id]), [
    :total_occupancy => latest_val => :total_occupancy,
    :icu_occupancy => latest_val => :icu_occupancy,
]);

In [9]:
capacity_rawdata = DataFrame(CSV.File("../data/capacity_hhs.csv"));

In [10]:
capacity_data = select(capacity_rawdata,
    :hospital_id,
    :capacity_icu => (x -> x .* 0.4) => :icu_beds,
    :capacity_allbeds => (x -> x .* 0.4) => :total_beds,
);

In [11]:
data_latest = rightjoin(data_latest, capacity_data, on=:hospital_id);

In [12]:
compute_load(a,b) = (a==0) ? 0.0 : (b==0) ? 1.0 : a/b;

In [13]:
insertcols!(data_latest, 5, :total_load => compute_load.(data_latest.total_occupancy, data_latest.total_beds));
insertcols!(data_latest, 8, :icu_load => compute_load.(data_latest.icu_occupancy, data_latest.icu_beds));

In [14]:
metadata = DataFrame(CSV.File("../data/hhs_hospital_meta.csv"));

In [15]:
hospital_locations_data = DataFrame(CSV.File("../data/hhs_hospital_locations.csv"))
select!(hospital_locations_data, :hospital_id, :lat, :long);

In [16]:
data_combined_all = leftjoin(data_latest, metadata, on=:hospital_id, makeunique=true);
data_combined_all = leftjoin(data_combined_all, hospital_locations_data, on=:hospital_id);

In [17]:
data_combined_all.lat = coalesce.(data_combined_all.lat, 0.0);
data_combined_all.long = coalesce.(data_combined_all.long, 0.0);

In [18]:
data_combined = select(data_combined_all,
    :hospitalname => :hospital,
    :hospital => :hhsname,
    :hospital_id,
    :lat, :long,
    :state, :state_abbrev, :city,
    :system_id, :system_name,
    :hsa_id, :hsa_name,
    :hrr_id, :hrr_name,
    :total_beds,
    :total_occupancy,
    :total_load,
    :icu_beds,
    :icu_occupancy,
    :icu_load,
)

,hospital,hhsname,hospital_id,lat,long,state,state_abbrev,city,system_id,system_name,hsa_id,hsa_name,hrr_id,hrr_name,total_beds,total_occupancy,total_load,icu_beds,icu_occupancy,icu_load
,String?,String?,String?,Float64,Float64,String?,String?,String?,String?,String?,Int64?,String?,Int64?,String?,Float64,Float64?,Float64,Float64,Float64?,Float64
1,Abbeville Area Medical Center,ABBEVILLE AREA MEDICAL CENTER,421301,34.1579,-82.3815,South Carolina,SC,Abbeville,missing,missing,42001,"Abbeville, SC",366,"Columbia, SC",6.4,5.6,0.875,0.0,0.0,0.0
2,Abbeville General Hospital,ABBEVILLE GENERAL HOSPITAL,190034,29.9737,-92.1077,Louisiana,LA,Abbeville,HSI00000562,Lafayette General Health,19001,"Abbeville, LA",213,"Lafayette, LA",15.2,5.1,0.335526,2.8,0.0,0.0
3,Abbott Northwestern Hospital,ABBOTT NORTHWESTERN HOSPITAL,240057,44.9526,-93.262,Minnesota,MN,Minneapolis,HSI00000029,Allina Health System,24076,"Minneapolis, MN",251,"Minneapolis, MN",266.0,50.9,0.191353,20.4,15.6,0.764706
4,Abilene Regional Medical Center,ABILENE REGIONAL MEDICAL CENTER,450558,32.3747,-99.744,Texas,TX,Abilene,HSI00000249,Community Health Systems,45001,"Abilene, TX",382,"Abilene, TX",32.8,32.9,1.00305,5.6,10.1,1.80357
5,Abington Memorial Hospital,ABINGTON MEMORIAL HOSPITAL,390231,40.1191,-75.1199,Pennsylvania,PA,Abington,HSI00000048,Jefferson Health,39001,"Abington, PA",356,"Philadelphia, PA",208.4,105.6,0.506718,22.0,16.6,0.754545
6,Abraham Lincoln Memorial Hospital,ABRAHAM LINCOLN MEMORIAL HOSPITAL,141322,40.1535,-89.39,Illinois,IL,Lincoln,HSI00000648,Memorial Health System,14075,"Lincoln, IL",172,"Springfield, IL",10.0,6.7,0.67,0.0,0.0,0.0
7,Abrazo Arrowhead Campus,ABRAZO ARROWHEAD CAMPUS,030094,33.6544,-112.202,Arizona,AZ,Glendale,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ",74.8,104.7,1.39973,12.0,25.9,2.15833
8,Abrazo Central Campus,ABRAZO CENTRAL CAMPUS,030030,33.5256,-112.102,Arizona,AZ,Phoenix,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ",43.6,64.3,1.47477,8.8,21.3,2.42045
9,Abrazo Scottsdale Campus,ABRAZO SCOTTSDALE CAMPUS,030083,33.6393,-111.998,Arizona,AZ,Phoenix,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ",24.0,43.1,1.79583,4.0,8.6,2.15


In [19]:
sort!(data_combined, [:hospital, :hospital_id]);

In [20]:
data_combined |> CSV.write("../data/hhs_current_load_covid.csv");

In [21]:
data_combined_list = collect([NamedTuple(h) for h in eachrow(data_combined)]);

In [22]:
serialize("../data/hhs_current_load_covid.jlser", data_combined_list)